In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, concat, col, lit
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark import SparkConf, SparkContext


import sys
from collections import defaultdict
import numpy as np
import re
import time

In [2]:
def process_str(row):

    body_list = []
    try:
        for word in row.lower().split(): 
            word = rgx.value.sub('', word)  
            body_list.append(word)
        return body_list
    except Exception as e:
        print(e)
        return ['']
process = udf(process_str, ArrayType(StringType()))

In [3]:

sc = SparkContext()
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
print('*'*60, '\n', sc.getConf().getAll(), '\n', '*'*60, '\n')


************************************************************ 
 [('spark.app.id', 'local-1563985731720'), ('spark.app.name', 'SimpleApp'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.driver.host', 'd6d187d3a6f1'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.port', '38697')] 
 ************************************************************ 



In [12]:
#Imports data into spark dataframe
#data = sc.textFile('s3://amazon-reviews-pds/amazon_reviews_us_Video_Games_v1_00.tsv.gz')
repartition_num = 10
data = sc.textFile('data/gaming_reviews.tsv')
df = spark.read.csv(data, sep="\t", header=True, inferSchema=True)
df = df.repartition(repartition_num)



In [13]:
#Subset selection for testing purposes
df = df.select('review_headline', 'review_body', 'star_rating').limit(100)

In [14]:
df.take(1)

[Row(review_headline='Great game very dynamic', review_body='Great game very dynamic.  If you are a fan of open world RPGs you will live this game.  My 7yo daughter loves this game, and I also appreciate it at nearly 40.  Reminds me of awesome games I played in my youth on SNES.', star_rating=5)]

In [6]:
df.take(1)

[Row(marketplace='US', customer_id=868003, review_id='RI5PHK7J8SLRF', product_id='B00IVHQ0KI', product_parent=442260730, product_title='Thrustmaster Ferrari 458 Spider Racing Wheel for Xbox One', product_category='Video Games', star_rating=5, helpful_votes=1, total_votes=2, vine='N', verified_purchase='Y', review_headline='Great entry level racing wheel for the price', review_body="Absolutely no regrets in regards to the purchase of this product, great entry level racing wheel if you don't want to fork over a great. I've never previously owned a racing wheel before so it is only my humble opinion, for what it's worth.  I've had it for about a month and though I had my doubts before purchasing it, it's quickly become a permanent fixture for me.  The steering is very precise and responsive as well as the foot pedals and paddle shifter.  My only issues with it, albeit very minor ones, would probably be the D-pad and the foot pedals could be spaced apart a bit more.", review_date=datetime.

In [ ]:
df_new = df.withColumn('body_list', process(df['review_body']))\
        .select('star_rating', 'body_list')

print("-"*30, '\nFinished creating new_df\n', "-"*30)
print(df_new.count())

cv = CountVectorizer(inputCol="body_list", outputCol="features")
print("created cv line 67")



In [ ]:
stage1_fit = cv.fit(df_new)
stage1_transform = stage1_fit.transform(df_new)
stage1_transform.repartition(repartition_num)